[Lecture 2: Recurrent Neural Networks and Transformers](https://www.youtube.com/watch?v=QvkQ1B3FBqA&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI)

goal: to make use of the fact that some data is sequential in making predictions.

it could be that our input is temporal or generally sequential, or that our output is.

the output at any given timestep depends on the input at that timestep.  
with sequential data, we also need to find a way to honor the fact that the output at t2 depends on the input at t1 and t0, not just at t2.

we do this by adding the concept of internal memory / state = h(t).  
our output depends not only on the input at that time, but on the memory from the prior time.  
$\hat{y}(t) = f(x_t, h_{t-1})$  
output depends on input and on past memory.  
cool!  

$h_t = f_W(x_t, h_{t-1})$

cell state is a function of weights, our current input, and our prior cell state.  
the weights are fixed for some reason...I suppose we only do gradient descent updating after a whole sequence of data is processed...we can't do training in the middle of some set of sequential data.  

^ is exactly right. we iterate over each piece (datum?) of the sequence of data before making a prediction. the sequence of data is sort of a single datum in this model. we use the recurrence relation to keep track of what's gone on since the beginning of the input. 

we have one weight matrix that we multiply by the past state, and another weight matrix that we multiply by the current input. we add the values of these matrix multiplication operations, take some sigmoid thing of them, and that gives our current state.

the same weight matrices are used at each time step.
we calculate the loss after each time step and sum them (but do NOT update the weight matrix on the basis of that loss calculation).

to model sequences, we need to:  
1. be able to deal with variable-length sequences.  
2. track long-term dependencies. (doesn't the model we get just track the immediate-past dependencies? it keeps track of the far past insofar as the immediate past is informed by the far past, but it does not keep trak of the far past itself. right? it would be easy to fix this by making our memory larger and making t_n a function of x_n, h_n-1, and h_n-1 [and so on if we want to keep track of longer-term dependencies].)  
3. maintain information about order. (this makes a lot of sense. in FFNNs it doesn't matter what order the data is fed in, which makes little sense in many contexts.)
4. share parameters across the sequence.  

RNNs meet all these criteria!

stopped at 21:06.